In [1]:
import time
from datetime import datetime
sc = SparkContext.getOrCreate()
#traces
import pandas as pd
import numpy as np
import re
import sys
import math
#for ordered dict for traces implementation
from collections import OrderedDict
#from pyspark.sql.types import VectorUDT
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

from datetime import datetime
from datetime import timedelta
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.types import DoubleType
#for timedelta manipulation
from math import fabs

from pyspark.sql import Row

#kmeans
#from pyspark.mllib.clustering import KMeans, KMeansModel
#from pyspark.mllib.linalg import Vectors
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from  pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors, VectorUDT


from pyspark.sql.functions import *
#from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import BisectingKMeans
from pyspark.sql import DataFrameReader
from pyspark.sql import DataFrame
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
#from pyspark.mllib.linalg import Vectors
from pyspark.sql import HiveContext
from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'



from pyspark import SparkContext
#sc =SparkContext()


pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#%load_ext autotime
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#del min
#del max

data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'

sqlContext = HiveContext(sc)



In [2]:
%load_ext autotime

In [3]:
def fix_df(df):
    return df.rdd.toDF(df.schema.names)


def change_column_names(df, old_names, new_names):
    pass
    return df;

time: 2.53 ms


In [4]:
#minMaxScaler wrapper since originalMin/Max is only implemented in 2.0
class scaler_wrapper():
    mmModel = ''
    originalMin = ''
    originalMax = ''   
    
    def __init__(self, inputCol, outputCol, s_min = 0, s_max = 0):
        self.mmModel = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)
        self.mmModel.setMin(s_min)
        self.mmModel.setMax(s_max)
        self.in_column = inputCol
        
    def get_input_col_name(self):
        return self.mmModel.getInputCol()

    def getMax(self):
        return self.mmModel.getMax()
        
    def getMin(self):
        return self.mmModel.getMin()
    
    def describe(self):
        print 'describe'
    
    def fit(self, df):
        col = self.mmModel.getInputCol()
        self.originalMin = df.select(col).rdd.flatMap(lambda x: x[0]).min()
        self.originalMax = df.select(col).rdd.flatMap(lambda x: x[0]).max()
        return self.mmModel.fit(df)
    
    #denormalize the value
    def denormalize(self, value):
        v = (value-self.getMin())*\
            (self.originalMax - self.originalMin)*\
            (self.getMax()-self.getMin()) + self.originalMin
        if v or v == 0:
            return v
        else:
            return -999
        
    def denormalize_df(self, df):
        col = self.mmModel.getInputCol()
        
        
    def normalize(self, value):
        pass

time: 26.5 ms


In [5]:
#calculate sse
def compute_sse(a,b):
    total = 0
    if len(a) != len(b):
        print 'bad input'
        return 99999
    else:
        for i in range(len(a)):
            total += math.pow(float(a[i])-float(b[i]),2)
    return math.sqrt(total)

def change_type_col_double(df, col):
    df = df.withColumn(col, df[col].cast("double"))
    return df.na.fill(0)

#change column type
def change_type_cols_double(df, list_cols):
    for col_name in list_cols:
        df = df.withColumn(col_name, df[col_name].cast("double"))
    return df.na.fill(0)

def normalize_vectorize_df(df, list_cols, scale_dict={}):
    n_df = vectorize_columns(df, list_cols)
    n_df, scalerModels = normalize_df(n_df, list_cols, scale_dict)
    return n_df, scalerModels
#vectorize the column
#keeps the original name
def vectorize_columns(df, list_cols, debug=False):
    a = datetime.now()
    tmp_col_name = 'temp'
    for col_name in list_cols:
        vectorize = udf(lambda vs: Vectors.dense(vs), VectorUDT())
        df = df.withColumn(tmp_col_name, vectorize(df[col_name])).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
    b = datetime.now()
    print 'vectorize done in', b-a
    return df

##scaler wrapper usage
#generate normalized dataframe
#keep the original column names
#returns:
#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df_old(df, list_cols, scale_dict = {}, debug=False):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        if debug:
            print '------- normalize ', col_name,'-------'
        a = datetime.now()
        scale_value = 1
        if col_name in scale_dict:
            scale_value = scale_dict[col_name]
            print scale_value
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
        scalerModel = scaler.fit(df)
        b = datetime.now()
        if debug:
            print b-a
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        c =datetime.now()
        if debug:
            print c-b
        index+=1
    return df, r_dict


#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df(df, list_cols, scale_dict = {}, debug=False):
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        if debug:
            print '------- normalize ', col_name,'-------'
        a = datetime.now()
        scale_value = 1
        if col_name in scale_dict:
            scale_value = scale_dict[col_name]
            print scale_value
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name, s_max = scale_value)
        scalerModel = scaler.fit(df)
        b = datetime.now()
        if debug:
            print b-a
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        c =datetime.now()
        if debug:
            print c-b
        index+=1
    return df, r_dict


# MinMaxScaler doesn't have originalMin(only supported in 2.0). made one above cell with wrapper class
# #generate normalized dataframe
# #keep the original column names
# #returns:
# #   (df, {col_name:scalerModel, col_name:scalerModel,....})
# def normalize_df(df, list_cols):
#     tmp_col_name = 'temp'
#     r_dict = {}
#     for col_name in list_cols:
#         scaler = MinMaxScaler(inputCol=col_name, outputCol=tmp_col_name)
#         scalerModel = scaler.fit(df)
#         df = scalerModel.transform(df).drop(col_name)\
#             .withColumnRenamed(tmp_col_name, col_name)
#         r_dict[col_name] = [scaler, scalerModel]
#     return df, r_dict

#input: (MLLIB KMEANS)
#   cols_to_cluster: list of column names to cluster
# def cluster_summary_df(df, cols_to_cluster, num_clusters = 2):
#     vecAssembler = VectorAssembler(inputCols=cols_to_cluster,\
#         outputCol='features')
#     df_to_cluster = vecAssembler.transform(df.na.fill(0))
#     df_to_cluster = df_to_cluster.select('features')
#     df_to_cluster = df_to_cluster.rdd\
#         .map(lambda row : Vectors.dense([item for item in row]))
#     clusters = KMeans.train(df_to_cluster, num_clusters,\
#                            maxIterations = 10,\
#                         initializationMode="random")
    
#     return clusters


#create feature column from list of cols
def create_feature_column(df, cols_to_cluster):
    #make dup columns of the ones that will be vectorized
    normalized_names = [i+'_n' for i in cols_to_cluster]
    
    norm_and_woutid = [i+'_n' for i in cols_to_cluster]
    norm_and_woutid.append('workoutid')

    normalized_df = df
    
    for c in range(len(cols_to_cluster)):
        normalized_df = normalized_df.withColumnRenamed(\
                        cols_to_cluster[c], normalized_names[c])
    normalized_df = normalized_df.select(norm_and_woutid)

    
    df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
                'inner').drop(normalized_df['workoutid'])
    #df = df.crossJoin(normalized_df)
    vecAssembler = VectorAssembler(inputCols=normalized_names,\
        outputCol='features')

    df_to_cluster = vecAssembler.transform(df.na.fill(0))

    #normed_df = df_to_cluster.select(normalized_names)
    return df_to_cluster   
    
    
#ML KMEANS
#returns KMeanModel, df_to_cluster
def cluster_summary_df(df, cols_to_cluster, num_clusters = 2, init_var=True, bisectingKMean = True,\
                      debug = False):
    a = datetime.now()

#     #make dup columns of the ones that will be vectorized
#     normalized_names = [i+'_n' for i in cols_to_cluster]
    
#     norm_and_woutid = [i+'_n' for i in cols_to_cluster]
#     norm_and_woutid.append('workoutid')
#     #make copy
#     a = datetime.now()
#     normalized_df = df
    
#     b = datetime.now()
#     print b-a , 'df copy'
    
#     for c in range(len(cols_to_cluster)):
#         normalized_df = normalized_df.withColumnRenamed(\
#                         cols_to_cluster[c], normalized_names[c])
#     normalized_df = normalized_df.select(norm_and_woutid)

    
#     df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
#                 'inner').drop(normalized_df['workoutid'])
#     #df = df.crossJoin(normalized_df)
#     vecAssembler = VectorAssembler(inputCols=normalized_names,\
#         outputCol='features')

#     df_to_cluster = vecAssembler.transform(df.na.fill(0))

#     normed_df = df_to_cluster.select(normalized_names)
    
    
#    df_to_cluster = df_to_cluster.select(['workoutid','features'])
    #df_to_cluster = df_to_cluster.rdd\
     #   .map(lambda row : Vectors.dense([item for item in row]))
    df_to_cluster = create_feature_column(df, cols_to_cluster)
    
        
        
    if not bisectingKMean:
        if not init_var:
            kmeans = KMeans(k=num_clusters, seed=1, initMode="random",\
                           featuresCol=cols_to_cluster)
        else:
            kmeans = KMeans(k=num_clusters, seed=1, initMode="k-means||")
    else:

        kmeans = BisectingKMeans().setK(num_clusters).setSeed(1).setMinDivisibleClusterSize(10)
    
    
    model = kmeans.fit(df_to_cluster)
    
    b = datetime.now()
    if debug:
        print 'clustering in ', b-a
    return model, df_to_cluster



# def cluster_summary_df(df, cols_to_cluster, num_clusters = 2, init_var=False):
#     #make dup columns of the ones that will be vectorized
#     normalized_names = [i+'_n' for i in cols_to_cluster]
    
#     norm_and_woutid = [i+'_n' for i in cols_to_cluster]
#     norm_and_woutid.append('workoutid')
#     #make copy
#     a = datetime.now()
#     normalized_df = df.copy()
    
#     b = datetime.now()
#     print b-a , 'df copy'
    
#     for c in range(len(cols_to_cluster)):
#         normalized_df = normalized_df.withColumnRenamed(\
#                         cols_to_cluster[c], normalized_names[c])
#     normalized_df = normalized_df.select(norm_and_woutid)

    
#     df = df.join(normalized_df, (df['workoutid']==normalized_df['workoutid']),\
#                 'inner').drop(normalized_df['workoutid'])
#     #df = df.crossJoin(normalized_df)
#     vecAssembler = VectorAssembler(inputCols=normalized_names,\
#         outputCol='features')

#     df_to_cluster = vecAssembler.transform(df.na.fill(0))

#     normed_df = df_to_cluster.select(normalized_names)
    
    
# #    df_to_cluster = df_to_cluster.select(['workoutid','features'])
#     #df_to_cluster = df_to_cluster.rdd\
#      #   .map(lambda row : Vectors.dense([item for item in row]))
        
#     print 'clustering times'
#     a = datetime.now()
#     if not init_var:
#         kmeans = KMeans(k=num_clusters, seed=1, initMode="random",\
#                        featuresCol=cols_to_cluster)
#     else:
#         kmeans = KMeans(k=num_clusters, seed=1, initMode="k-means||")
#     b = datetime.now()
#     print b - a
    
#     model = kmeans.fit(df_to_cluster)
    
#     c = datetime.now()
#     print c - b
#     return model, df_to_cluster

    
#returns clusters. show cluster centerse by clusters.clusterCenters.
#output:
#   [0]: KMeanModel
#   [1]: DF that was clustered (normalized)
#   [2]: ScalerModels
#   [3]: error
def cluster_df(df, cols_to_normalize, num_clusters = 2, return_sse = False, cols_to_cluster = [], init_var=True,\
              scale_dict = {}, bisectingKMean = True, debug = False):
    if not cols_to_cluster:
        cols_to_cluster = cols_to_normalize
    df = change_type_cols_double(df, cols_to_normalize)
    df = vectorize_columns(df, cols_to_normalize, debug)
    df, scalerModels = normalize_df(df, cols_to_cluster, scale_dict, debug)
    r_obj = cluster_summary_df(df, cols_to_cluster, num_clusters, init_var, bisectingKMean, \
                              debug)
    sse = 'none'
    if return_sse:
        #calculate SSE for this kmean model
        kmodel = r_obj[0]
        df_to_cluster = r_obj[1]
        centers = kmodel.clusterCenters()
        prediction_df = kmodel.transform(df_to_cluster)

        #set up rdd
        rdd = prediction_df.rdd.map(lambda (a,b):\
                                   (a, centers[b]))
        sse = rdd.map(lambda (a,b): compute_sse(a,b)).sum()
    
    return r_obj[0], r_obj[1], scalerModels, sse

def denormalize_centers(list_centers, scaler_models):
    new_centers = []
    for center in list_centers:
        n_center = []
        for c in range(len(center)):
            n_center.append(scaler_models[c].denormalize(center[c]))
        new_centers.append(n_center)
    return new_centers



def denormalize_cluster(cluster_centers, list_scaler_models):
    r_centers = []
    for center in cluster_centers:
        r_center = []
        for index in range(len(center)):
            s_model = list_scaler_models[index]
            r_center.append(s_model.denormalize(center[index]))
        r_centers.append(r_center)
    return r_centers

def get_distribution_of_workouts(transformed_df):
    #count of predictions
    return transformed_df.groupBy('prediction').count().orderBy('prediction')

#transform (add prediction column) to the dataframe based on the kmeamodel input
def transform_df_with_kmeamodel(kmeanModel, df, cols_to_cluster, scale_dict={}):
    #need to vectorize and normalize the df
    df_to_transform, scalerModels = normalize_vectorize_df(big_sumstats, cols_to_cluster, scale_dict)
    df_to_transform = create_feature_column(df_to_transform, cols_to_cluster)
    return kmeanModel.transform(df_to_transform), scalerModels



#
def vectorizeData(data):
    return data.rdd.map(lambda r: [r[0], r[1], r[2], Vectors.dense(r[3:6]), r[7]]).toDF([\
            'prediction', 'userid', 'workoutid', 'label', 'max_elapsed_time'])

# total_df= fix_df(sumstats.join(prediction_df.select('workoutid','prediction'), \
#                                sumstats['workoutid']==prediction_df['workoutid'],'inner').\
#                             drop(prediction_df['workoutid']))




time: 672 ms


In [6]:
from pyspark.sql import DataFrameReader

url = 'postgresql://172.31.29.1:5432/endomondo_sample'
#url = 'postgresql://172.31.29.1:5432/endomondo'

properties = {'user':'endomondo', 'password': 'End0m0ndo'}

df_runs = DataFrameReader(sqlContext).jdbc(url='jdbc:%s' % url, table='run', properties=properties)

df_users = DataFrameReader(sqlContext).jdbc(url='jdbc:%s' % url, table='run_by_workout', properties=properties)

df_runs

DataFrame[altitude: decimal(20,10), heart_rate: decimal(10,5), latitude: decimal(20,10), longitude: decimal(20,10), speed: decimal(20,10), workoutid: int, time: int, altitude_first: decimal(10,5), altitude_second: decimal(10,5), elapsed_time: int, speed_first: decimal(10,5), geo_distance: decimal(20,10), elapsed_distance: decimal(15,10), heart_rate_ma_25: decimal(8,5), speed_ma_100: decimal(8,5), speed_ma_50: decimal(8,5), speed_by_geo: decimal(20,10)]

time: 2.08 s


In [ ]:
df_users.count()

In [8]:
sub_df = df_runs.limit(1000)
df_users = df_users.withColumn('diff_altitude', \
                    df_users['altitude_max'] \
                    -df_users['altitude_min'])
sumstats = df_users.limit(100)#.limit(1000)

time: 116 ms


In [9]:
route_cols = ['diff_altitude', 'geo_distance']


time: 897 µs


In [10]:
scale_dict = {}

r_obj = cluster_df(sumstats, route_cols, 5, init_var=True,\
                  scale_dict=scale_dict, bisectingKMean=True,\
                  debug=True)
print 'done'

vectorize done in 0:00:00.129901
------- normalize  diff_altitude -------


KeyboardInterrupt: 

time: 1min 51s


In [244]:
route_kmodel = r_obj[0]
route_df = r_obj[1]
route_scalerModels = r_obj[2]

time: 2.02 ms


In [245]:
###check if routes make sense

time: 711 µs


In [218]:
route_prediction_df = route_kmodel.transform(route_df)
route_prediction_cols = ['route_prediction','route_features']
distinct_clusters = route_prediction_df.select('prediction').distinct().collect()
route_prediction_df = route_prediction_df.withColumnRenamed('prediction', 'route_prediction')
route_prediction_df = route_prediction_df.withColumnRenamed('features', 'route_features')



time: 311 ms


In [219]:
route_prediction_df.groupBy('route_prediction').count().show()
route_prediction_df = fix_df(route_prediction_df)

+----------------+-----+
|route_prediction|count|
+----------------+-----+
|               2|  152|
|               1|  327|
|               0|  358|
|               3|  151|
|               4|   12|
+----------------+-----+

time: 1.64 s


In [220]:
route_kmodel.clusterCenters()

[array([ 0.00561581,  0.08662023]),
 array([ 0.00825486,  0.14370067]),
 array([ 0.01178036,  0.19603238]),
 array([ 0.01898868,  0.28943591]),
 array([ 0.03199172,  0.68608366])]

time: 7.82 ms


In [221]:
denormalize_cluster(route_kmodel.clusterCenters(),  route_scalerModels)

[[34.337319662011183, 1601.0664774709555],
 [50.473503174311958, 2656.1269716579118],
 [72.0298458157895, 3623.4131298017123],
 [116.1043967549669, 5349.8604437707518],
 [195.610175, 12681.397421410449]]

time: 6.86 ms


In [222]:
distribution_df = get_distribution_of_workouts(route_kmodel.transform(route_df))
distribution_df.show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0|  358|
|         1|  327|
|         2|  152|
|         3|  151|
|         4|   12|
+----------+-----+

time: 313 ms


In [223]:
#######################end route checker

time: 712 µs


In [246]:
#get cluster numbers
cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]
cluster_numbers

[2, 1, 0, 3, 4]

time: 3.4 ms


In [247]:
#create list of dataframes for each route_prediction value
list_route_clusters = [route_prediction_df.where(route_prediction_df['route_prediction']==i) \
                      for i in cluster_numbers]

time: 20.5 ms


In [249]:
#got list of dataframes by route cluster. for each of them, create new clusters 
#add a new column 'performance_prediction' for each cluster

performance_cols_to_cluster = ['heart_rate_avg', 'speed_avg', 'elapsed_time']

dict_cluster_to_objs = {}

n_list_route_clusters = []

for i in range(len(list_route_clusters)):
    print i, 
#     #cluster
#     c_df = list_route_clusters[i]
#     r_obj = cluster_df(c_df, performance_cols_to_cluster, 5, init_var=True, \
#                       scale_dict = scale_dict, bisectingKMean=True,\
#                       debug=False)
#     dict_cluster_to_objs[i] = r_obj

#     #transform the dataset
#     p_kmodel = r_obj[0]
#     p_df_to_transform = r_obj[1]
#     p_scalerModels = r_obj[2]
    
#     p_kmodel.transform(p_df_to_transform)
    c_df = list_route_clusters[i]
    #cluster within the route cluster dataframe
    r_obj = cluster_df(c_df, performance_cols_to_cluster, 3, init_var=True, \
                      scale_dict = scale_dict, bisectingKMean=True,\
                      debug=False)
    dict_cluster_to_objs[i] = r_obj

    #transform the dataset
    p_kmodel = r_obj[0]
    p_df_to_transform = r_obj[1]
    p_scalerModels = r_obj[2]
    print 'transforming'
    temp_df = p_kmodel.transform(p_df_to_transform)
    temp_df = temp_df.withColumnRenamed('features', 'perf_features')
    temp_df = temp_df.withColumnRenamed('prediction', 'perf_prediction')
    #print c_df.schema
    print '\n'
    #transform
    c_df = c_df.drop('perf_features')
    c_df = c_df.drop('perf_prediction')
    a = datetime.now()

    #list_route_clusters[i] = c_df.join(temp_df.select('workoutid','perf_prediction', 'perf_features'),\
    #                                  temp_df['workoutid']==c_df['workoutid'], 'inner')\
    #                                    .drop(temp_df['workoutid'])
    n_list_route_clusters.append(c_df.join(temp_df.select('workoutid','perf_prediction', 'perf_features'),\
                                      temp_df['workoutid']==c_df['workoutid'], 'inner')\
                                        .drop(temp_df['workoutid']))
    b = datetime.now()
    print 'joined', b-a
    #list_route_clusters[i].show()
    #break
    


0 vectorize done in 0:00:00.067778
transforming


joined 0:00:00.055359
1 vectorize done in 0:00:00.072193
transforming


joined 0:00:00.056016
2 vectorize done in 0:00:00.073228
transforming


joined 0:00:00.061502
3 vectorize done in 0:00:00.072374
transforming


joined 0:00:00.055361
4 vectorize done in 0:00:00.068054
transforming


joined 0:00:00.055101
time: 1min 51s


In [250]:
prediction_columns = ['route_prediction', 'perf_prediction']
frames = [df.toPandas() for df in n_list_route_clusters]

time: 7.17 s


In [251]:
new_concat_df = sqlContext.createDataFrame(pd.concat(frames))

time: 408 ms


In [252]:
new_concat_df.groupBy(['route_prediction','perf_prediction']).count()\
.orderBy(['route_prediction', 'perf_prediction']).show()

+----------------+---------------+-----+
|route_prediction|perf_prediction|count|
+----------------+---------------+-----+
|               0|              0|   32|
|               0|              1|  229|
|               0|              2|   97|
|               1|              0|   44|
|               1|              1|  181|
|               1|              2|  102|
|               2|              0|   13|
|               2|              1|   98|
|               2|              2|   41|
|               3|              0|   11|
|               3|              1|   82|
|               3|              2|   58|
|               4|              0|    8|
|               4|              1|    4|
+----------------+---------------+-----+

time: 315 ms


In [253]:
#remove small clusters, might be outliers
new_concat_df = new_concat_df.where(
    ((new_concat_df['route_prediction']!= 3) | (new_concat_df['perf_prediction']!= 0))&\
    ((new_concat_df['route_prediction']!= 4) | (new_concat_df['perf_prediction']!= 0))&\
    ((new_concat_df['route_prediction']!= 4) | (new_concat_df['perf_prediction']!= 1))\
)

time: 10.1 ms


In [254]:
select_cols = prediction_columns+route_cols+['userid','workoutid']+performance_cols_to_cluster
print select_cols

['route_prediction', 'perf_prediction', 'diff_altitude', 'geo_distance', 'userid', 'workoutid', 'heart_rate_avg', 'speed_avg', 'elapsed_time']
time: 1.47 ms


In [255]:
sum_cols = route_cols+performance_cols_to_cluster+['workoutid']


time: 1.09 ms


In [259]:
#magic
unvector = udf(lambda value: value[0].item(), FloatType())

time: 7.09 ms


In [262]:
f_cluster_df = new_concat_df.select(select_cols)
f_cluster_df = f_cluster_df.withColumn('diff_altitude', unvector(f_cluster_df['diff_altitude']))
f_cluster_df = f_cluster_df.withColumn('geo_distance', unvector(f_cluster_df['geo_distance']))



time: 23.5 ms


In [263]:
f_cluster_df.write.csv('f_cluster.csv')

time: 910 ms


In [56]:
print c

None
time: 1.19 ms
